<a href="https://colab.research.google.com/github/kangmg/personal_study/blob/main/lmdb_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q ase lmdb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 19.2 MB/s eta 0:00:00


In [13]:
#@title Download dataset | SAMSUNG AI CHALLENGE 대회 데이터셋

# format : extxyz

!wget -q 'https://drive.usercontent.google.com/download?id=1whTYsrijTuNFoV8ATn-L__Tb9WDsYRnm&export=download&authuser=1&confirm=t' -O dataset.xyz


In [64]:
# 데이터셋 로드

from ase.io import read
import ase
import json
import numpy as np
from tqdm import tqdm
import lmdb

dataset = read('dataset.xyz', format='extxyz', index=':')

In [48]:
print(f"Dataset Size : {len(dataset)}")
print(f"Unique Num. Atoms : {set({len(atoms) for atoms in dataset})}") # All crystals are 96 size unit cell

Dataset Size : 1500
Unique Num. Atoms : {96}


In [94]:
# lmdb 데이터셋으로 변환

atoms = dataset[0]

datapoint = {
    **atoms.calc.results,
    'atomic_numbers': atoms.get_atomic_numbers(),
    'positions': atoms.positions,
    'cell': atoms.cell,
    'pbc': atoms.pbc,
}

datapoint.keys()

dict_keys(['forces', 'energy', 'stress', 'free_energy', 'atomic_numbers', 'positions', 'cell', 'pbc'])

In [92]:
def _np2list(np_array)->list:
    """convert np.array to list
    """
    if isinstance(np_array, np.ndarray):
        return np_array.tolist()
    elif isinstance(np_array, ase.cell.Cell):
        return np_array[:].tolist()
    else:
        return f"######{type(np_array)}######"

In [95]:
# 예시 데이터 인코딩


json.dumps(datapoint, default=_np2list).encode('utf-8')

b'{"forces": [[0.08813055, -0.90894865, 1.04011568], [0.49469689, -0.23481429, 1.14418526], [-0.47646964, 1.67774442, -1.52065335], [0.70496183, 1.83900631, 1.37385827], [-0.09108712, -1.1645404, 1.44755996], [-0.58542337, -1.35677337, 0.18130858], [0.07428873, 0.18930049, -0.34747464], [0.27813937, -1.26045232, 0.05925787], [-0.31964407, 0.46092358, 0.87237368], [-1.06522241, 0.08927455, -1.37828529], [0.73032466, -0.6762505, -0.80599051], [1.79984528, -0.53344427, -1.28798645], [-0.53339657, 2.94097424, -1.45533789], [0.3137304, 0.42305567, -0.3276734], [-0.08893817, -0.68071368, 2.05788164], [-1.54500838, -0.73283706, -0.60443396], [0.332729, 0.23069284, -0.05415299], [1.23065039, -2.84575677, 1.16380071], [1.34493804, 0.27764302, 0.04367145], [-0.31247925, 0.69163507, -1.19526773], [-0.25171167, 0.87168053, 1.64834181], [0.65795932, 2.25250977, 0.65062972], [0.91047539, -3.24651499, 0.80779024], [-1.34380579, 2.54150941, -0.43351624], [-0.30848146, -0.31742813, -0.70369804], [0.444

In [86]:
def mb_to_bytes(mb:int=1024):
    """
    convert mb to bytes
    Default : 1024 MB (1GB)
    """
    print(f"LMDB : {mb} MB configured !")
    return int(mb * 1024 * 1024)

lmdb_env = lmdb.open('dataset.lmdb', map_size=mb_to_bytes(1024))

with lmdb_env.begin(write=True) as txn:
    for idx, datapoint in enumerate(tqdm(dataset)):
        data = {
            **datapoint.calc.results,
            'atomic_numbers': datapoint.get_atomic_numbers(),
            'positions': datapoint.positions,
            'cell': datapoint.cell,
            'pbc': datapoint.pbc
            }
        data_encoded = json.dumps(data, default=_np2list).encode('utf-8')
        txn.put(f"{idx}".encode('utf-8'), data_encoded)

LMDB : 1024 MB configured !


100%|██████████| 1500/1500 [00:00<00:00, 2017.20it/s]


In [87]:
# read lmdb DB
def dataloader(lmdb_path:str):
    open_env = lmdb.open(lmdb_path, readonly=True)
    with open_env.begin(write=False) as txn:
        for key, value in txn.cursor():
            yield json.loads(value.decode('utf-8'))

dl = dataloader('dataset.lmdb')

In [91]:
# 예시로 에너지 10개 출력
for i in range(10):
    print(next(dl)['energy'])

-927.84188598
-893.87290297
-952.09046603
-937.79367081
-891.64564392
-876.48700441
-906.66619234
-883.09227724
-917.56554749
-900.35989411
